# H20.AI website
---
This notebook builds a flask model to deploy the **`Kanmani`  ANN model** into a localhost website 

# History

The `kanmani` model, an integration of the `LSTM`-`CNN1D` ensembled submodels built for 4 reservoirs, is made to learn from the 2004-2018 reservoir datasets, and is tested against the original outflow. After coining the model as **perfect**, we tend to move to post-production, ie, towards launching our model in the website

## STEP 0: Prepare essentials
1. Import libraries
2. Import Kanmani ANN model
3. Check model summary

In [1]:
from flask import Flask, redirect, url_for, request
import pandas as pd
import tensorflow as tf
import numpy as np
from datetime import datetime  
from tensorflow.keras import models
import requests
from datetime import datetime  
from datetime import timedelta  

modelv2=models.load_model('V-D-AL.h5')
print('Done importing essentials')
dictionary={
    'Chembarambaakam':0,
    'Chozhavaram':1,
    'Poondi':2,
    'RedHills':3
}
revdict={
   
    0:'Chembarambaakam',
    1:'Chozhavaram',
    2:'Poondi',
    3:'RedHills'
}
dianna=[]
HOMEDIR='D:\Semester 7\TARP\datafolder\\'
dianna.append(pd.read_csv(HOMEDIR+'CHdianna2.csv',na_values=-1))
dianna.append(pd.read_csv(HOMEDIR+'CZdianna2.csv',na_values=-1))
dianna.append(pd.read_csv(HOMEDIR+'PNdianna2.csv',na_values=-1))
dianna.append(pd.read_csv(HOMEDIR+'RHdianna2.csv',na_values=-1))
for i in range(0,4):
    print('\n\n',revdict[i],':\n')
    dianna[i]=dianna[i].dropna()
    dianna[i].rename(columns={"Storage (mcft)":"Water Level (mcft)"},inplace=True)
    dianna[i]=dianna[i].reset_index(drop=True)
    dianna[i]=dianna[i].set_index('Date')
    print(dianna[i].tail())

C:\Users\R. Senthil Kumar\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\R. Senthil Kumar\anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
C:\Users\R. Senthil Kumar\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


Done importing essentials


 Chembarambaakam :

            nth day in year  Water Level (mcft)  Inflow (cusecs)  \
Date                                                               
27/12/2019              360              1689.0            402.0   
28/12/2019              361              1718.0            402.0   
29/12/2019              362              1712.0              0.0   
30/12/2019              363              1706.0              0.0   
31/12/2019              364              1700.0              0.0   

            Rainfall (mm)  Outflow (cusecs)  
Date                                         
27/12/2019            0.0              67.0  
28/12/2019            0.0              67.0  
29/12/2019            0.0              67.0  
30/12/2019            0.0              67.0  
31/12/2019           13.0              67.0  


 Chozhavaram :

            nth day in year  Water Level (mcft)  Inflow (cusecs)  \
Date                                                               

In [2]:
##TODEL
import pickle
modelv2=[]
for i in range(4):
    reg = pickle.load(open(HOMEDIR+'versailles_'+str(i)+'.sav', 'rb'))
    modelv2.append(reg)

In [3]:
LOOKBACK=30
LOOKFRONT=15

# STEP 1: Lay foundations for the `docs` website
Write HTML-CSS-JS code for the ML-integrated website

In [4]:
docs='<!DOCTYPE html><html><head><title>H20.AI | Home</title><meta charset="UTF-8"><link rel="icon" href="https://img.icons8.com/pastel-glyph/2x/water.png"><meta name="viewport" content="width=device-width, initial-scale=1"></head><link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css"><link rel="stylesheet" href="https://fonts.googleapis.com/css?family=Raleway"><link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css"><style>html,body,h1,h2,h3,h4,h5{font-family:"Raleway", sans-serif}</style><style>.loader{border:16px solid #f3f3f3;border-radius:50%;border-top:16px solid blue;border-bottom:16px solid blue;width:120px;height:120px;-webkit-animation:spin 2s linear infinite;animation:spin 2s linear infinite}@-webkit-keyframes spin{0%{-webkit-transform:rotate(0deg);}100%{-webkit-transform:rotate(360deg);}}@keyframes spin{0%{transform:rotate(0deg);}100%{transform:rotate(360deg);}}</style><body style="min-height:93vh;background-color:#EEEEEE"><div class="w3-bar w3-top w3-black w3-large" style="z-index:4"><button class="w3-bar-item w3-button w3-hide-large w3-hover-none w3-hover-text-light-grey" onclick="w3_open();"><i class="fa fa-bars"></i> Menu</button><span class="w3-bar-item w3-right">|<- H20.AI ->|</span></div><div class="w3-overlay w3-hide-large w3-animate-opacity" onclick="w3_close()" style="cursor:pointer" title="close side menu" id="myOverlay"></div><div class="w3-light-grey" class="w3-main" style="margin-top:43px;margin-left:15%;margin-right:15%;background-color:#DDDDDD"><div><br><br><br><h3 style="margin-top:10px;margin-left:1%"><b><strong>H20.AI: </strong></B> Chennai\'s very own AI Water Supply Manager</h3><p style="width:75%;margin-top:10px;margin-left:1%">The project, built for TARP by Somya and Senthil, focuses on implementing Mathematics, Machine Learning and Artificial Intelligence to manage outflow of water from these 4 reservoirs that supply water to chennai. We will use this project as a medium to address the Chennai water crisis and mitigate its effects.<br>Please be patient while the page loads for 5-6 seconds</p><br><h5 style="width:75%;margin-top:10px;margin-left:1%">Enter a <b>valid</b> date in dd/mm/yyyy format to forecast outflow</h5><form id="ABC" action="" method="GET"><input id= "sawd" style="background-color:#EEEEEE;margin-left:1%;margin-right:1px" name="nm" type="text" size="40%" placeholder="Enter a date in dd/mm/yyyy format"/><button style="color:#FFFFFF;background-color:#000000"><b> Forecast! </b></button></form></div><br><header class="w3-container" style="padding-top:22px"><h5><b><i class="fa fa-dashboard"></i>   Forecasted Outflow: <span id="o">NaN</span></b></h5></header><div class="w3-row-padding w3-margin-bottom"><div class="w3-quarter"><div class="w3-container w3-red w3-padding-16"><div class="w3-left"></div><div class="w3-clear"></div><h4><b>Chembarambaakam</b></h4><div class="w3-right"><h3><span id="a">-1</span> cusecs</h3></div></div></div><div class="w3-quarter"><div class="w3-container w3-blue w3-padding-16"><div class="w3-left"></div><div class="w3-clear"></div><h4><b>Cholavaram</b></h4><div class="w3-right"><h3><span id="b">-1</span> cusecs</h3></div></div></div><div class="w3-quarter"><div class="w3-container w3-teal w3-padding-16"><div class="w3-left"></div><div class="w3-clear"></div><h4><b>Poondi</b></h4><div class="w3-right"><h3><span id="c">-1</span> cusecs</h3></div></div></div><div class="w3-quarter"><div class="w3-container w3-orange w3-text-white w3-padding-16"><div class="w3-left"></div><div class="w3-clear"></div><h4><b>Redhills</b></h4><div class="w3-right"><h3><span id="d">-1</span> cusecs</h3></div></div></div></div><hr><br><div id="aqastas" align="center"style="padding: 10px; padding-bottom:20px; background-color: white; border: 2px dashed black;position: absolute;left:45%;bottom:45%;"><h3>Forecasting...</h3><div class="loader"></div></div></div><div class="w3-container w3-dark-grey w3-padding-32"><div class="w3-row"><div class="w3-container w3-third"><h5 class="w3-bottombar w3-border-green"><strong>©2021</strong></h5></div><div class="w3-container w3-third"><h5 class="w3-bottombar w3-border-red">ML + we = ♥</h5></div><div class="w3-container w3-third"><h5 class="w3-bottombar w3-border-orange">Contact: Somya Sheti and Senthil Kumar</h5></div></div></div><script>;</script><script src="//ajax.googleapis.com/ajax/libs/jquery/1.9.1/jquery.min.js"></script></body><script>;$(document).ready(function(){$(\'#aqastas\').fadeOut(1);var a;$(\'#ABC\').submit(function(t){t.preventDefault();if(a){a.abort()};var s=$(this);$(\'#aqastas\').fadeIn("slow");var l=s.find(\'input, select, textarea\'),e=s.serialize();console.log(e);$.get(\'/_cal\',e,function(a){console.log(\'Response: \'+a);res=a.split(\',\');$(\'#o\').html($(\'#sawd\').val());$(\'#a\').html(res[1]);$(\'#b\').html(res[2]);$(\'#c\').html(res[3]);$(\'#d\').html(res[4]);$(\'#aqastas\').fadeOut("slow");console.log(res[5]);if(res[5]===\'F\')alert(\'Reservoirs are reaching near-full capacity, possible chance of a flood if left unchecked, please stay alert and follow meteorological reports\');else if(res[5]===\'D\')alert(\'Reservoir tanks are nearing dryout, Possible chance of a drought, please save water and use minimal amount\');})})});</script></html>'
print('Raw HTML ready')

Raw HTML ready


# STEP 2: Build the `flask` module
The `flask` module gets date from website, and generates optimal outflow for the given date using the **`Kanmani` ANN model**.

In [5]:
def findoccurence(n,content):

    position=0
    for i in range(0,n):
        temp=content[content.find('<td ',position):content.find('</td>',position)]
        #print(temp)
        temp=temp[temp.rfind('>',0,temp.find('</font>')-1)+1:temp.find('</font>')]
        #(temp)

        position=content.find('</td>',position)+1
    try:
        float(temp)
        return temp
    except:
        return -1

def ceil(x):
        if(x<0):
            return 0
        if(x-x//1>0):
            return x//1+1
        else:
            return x
        
def flood_or_drought(lisres):
    storagefull=[3645,881,3231,3300]
    for i in range(4):
        if(lisres[i]>=0.9*storagefull[i]):
            return "F"
        if(lisres[i]<=0.5*storagefull[i]):
            return "D"
    return "N"

In [ ]:
app = Flask(__name__)
content=''


@app.route('/')
def index():
    return docs


@app.route('/_cal')
def runner():
    start=0
    lisres=list()
    required_td=[14,15,17,16,23,24,26,25,32,33,35,34,59,60,62,61]
    sdate = request.args.get('nm')
    matrix=list()
    x=datetime.strptime(sdate,'%d/%m/%Y')
    y=x-timedelta(days=LOOKFRONT)
    x=x-timedelta(days=LOOKBACK+LOOKFRONT)

    edate=y.strftime("%d")+'/'+y.strftime("%m")+'/'+y.strftime("%Y")
    stringer=x.strftime("%d")+'/'+x.strftime("%m")+'/'+x.strftime("%Y")
    #print('target: ',edate,', start date: ',stringer)

    controller=1
    #print('Deploying SPYder, please wait...\n(do not disconnect the net-connection or abort the process)\n')
    while(stringer!=edate):
        print('current SPYder rezendevous: '+stringer)
        diff=(x-datetime(int(x.strftime("%Y")),1,1)).days
        matrix.append([diff])
        pointer=2
        parameters = {'ldate':stringer}
        r = requests.post('http://123.63.203.150/reserve3.asp', data=parameters)
        content=r.text
        origin=1
        for i in required_td:
            matrix[controller-1].append(float(findoccurence(i,content)))
            pointer+=1
            if((pointer-1)%5==0):
                origin+=1
                if(origin==5):
                    break
                else:
                    controller+=1
                    matrix.append([diff])
                    pointer=2

        x+=timedelta(days=1)
        stringer=x.strftime("%d")+'/'+x.strftime("%m")+'/'+x.strftime("%Y")
        controller+=1

    
    #This part is to swap positions of reservoir data matrix values, because our model was built with CH-CZ-PN-RH dataset order
    #Whereas, the mined data gives us PN-CZ-RH-CH order
    for mdays in range(0,LOOKBACK):
        temp=matrix[mdays*4]
        matrix[mdays*4]=matrix[mdays*4+3]
        matrix[mdays*4+3]=matrix[mdays*4+2]
        matrix[mdays*4+2]=temp

    #print(matrix)

    starter=0
    np_arr=[]
    for reservoir_index in range(4):
        curarr=[matrix[starter+day*4+reservoir_index] for day in range(LOOKBACK)]
        tafler=[]
        for day in curarr:
            for data in day:
                tafler.append(data)
        #print(tafler)
        #print(modelv2[reservoir_index].predict([tafler])[0])
        lisres.append(modelv2[reservoir_index].predict([tafler])[0])
        np_arr.append(np.array([curarr]))

    '''
    r=modelv2.predict([np_arr[reservoir_index] for reservoir_index in range(4)])

    r=modelv2.predict([np.array([[matrix[starter],matrix[4+starter],matrix[8+starter],matrix[12+starter],matrix[16+starter]]]),
                       np.array([[matrix[starter+1],matrix[4+starter+1],matrix[8+starter+1],matrix[12+starter+1],matrix[16+starter+1]]]),
                       np.array([[matrix[starter+2],matrix[4+starter+2],matrix[8+starter+2],matrix[12+starter+2],matrix[16+starter+2]]]),
                       np.array([[matrix[starter+3],matrix[4+starter+3],matrix[8+starter+3],matrix[12+starter+3],matrix[16+starter+3]]])])
    lisres=list()
    for ias in r:
        lisres.append(ceil(ias[0][0]))
        '''


    lisres2=[ceil(x) for x in lisres]

    #to prepare a return sequence : from [a,b,c,d] model output to "a,b,c,d" string for the website

    
    resultino="";
    for i in lisres2:
        resultino+=(","+str(i))
    
    resultino+=","+flood_or_drought(lisres2)
    print("Output passed:",resultino)
    return resultino

if __name__ == '__main__':
       app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Dec/2021 11:54:33] "GET / HTTP/1.1" 200 -


current SPYder rezendevous: 21/03/2019
current SPYder rezendevous: 22/03/2019
current SPYder rezendevous: 23/03/2019
current SPYder rezendevous: 24/03/2019
current SPYder rezendevous: 25/03/2019
current SPYder rezendevous: 26/03/2019
current SPYder rezendevous: 27/03/2019
current SPYder rezendevous: 28/03/2019
current SPYder rezendevous: 29/03/2019
current SPYder rezendevous: 30/03/2019
current SPYder rezendevous: 31/03/2019
current SPYder rezendevous: 01/04/2019
current SPYder rezendevous: 02/04/2019
current SPYder rezendevous: 03/04/2019
current SPYder rezendevous: 04/04/2019
current SPYder rezendevous: 05/04/2019
current SPYder rezendevous: 06/04/2019
current SPYder rezendevous: 07/04/2019
current SPYder rezendevous: 08/04/2019
current SPYder rezendevous: 09/04/2019
current SPYder rezendevous: 10/04/2019
current SPYder rezendevous: 11/04/2019
current SPYder rezendevous: 12/04/2019
current SPYder rezendevous: 13/04/2019
current SPYder rezendevous: 14/04/2019
current SPYder rezendevou

127.0.0.1 - - [06/Dec/2021 11:55:10] "GET /_cal?nm=05%2F05%2F2019 HTTP/1.1" 200 -


Output passed: ,20.0,14.0,211.0,137.0,D
